In [24]:
import os
import numpy as np
import pickle
import torch
from utils import load_model
from problems.tsp.problem_tsp import TSPDataset
import time
from mcts.mcts import MCTS_TSP
from mcts.mcts_utils import evaluate_tour

In [2]:
### PARAMETERS TO BE SET ###
n_graphs = 10
n_nodes = 20

n_expansions = 100
n_rollouts = 100
eval_selection = "best"
eval_rollout = "best"

temperature = 1

In [25]:
# Load model 20 nodes and set temperature
model, _ = load_model(F'pretrained/tsp_{n_nodes}/')
model.eval()
print("model loaded")
# Load test set graphs 20 nodes
# If this block does not work, make sure you called:
# python generate_data.py --problem all --name test --seed 1234
with open(F"data/tsp/tsp{n_nodes}_test_seed1234.pkl", "rb") as f:
    data = pickle.load(f)
    dataset = TSPDataset(None, 0, 0, 0, None)
    dataset.data = [torch.FloatTensor(row) for row in (data[0:0+10000])]
    dataset.size = len(dataset.data)
    graphs = []
    for sample in dataset.data:
        graphs.append(sample)

  [*] Loading model from pretrained/tsp_20/epoch-99.pt
model loaded


In [26]:
total_len = 0
total_len_best_seen = 0

timestamps = []
results = []

for i in range(n_graphs):
    graph = graphs[i]
    graph_batch = graph[None] # Add batch dimension
    tour = [0] # Start at first node, unconventional, TODO: fix this
    t_s = time.perf_counter()
    with torch.no_grad():
        embeddings, _ = model.embedder(model._init_embed(graph_batch))

        # Compute keys, values for the glimpse and keys for the logits once as they can be reused in every step
        fixed = model._precompute(embeddings)
        for visit in range(graph.shape[0] - 1):
            tour_tensor = torch.tensor(tour).long()
            if len(tour_tensor) == 0:
                step_context = model.W_placeholder
            else:
                step_context = torch.cat((embeddings[0, tour_tensor[0]],
                                        embeddings[0, tour_tensor[-1]]), -1)
            query = fixed.context_node_projected + model.project_step_context(step_context[None, None, :])
            mask = torch.zeros(graph_batch.shape[1], dtype=torch.uint8) > 0
            mask[tour_tensor] = 1
            mask = mask[None, None, :]

            log_p, _ = model._one_to_many_logits(query, fixed.glimpse_key, fixed.glimpse_val, fixed.logit_key, mask)
            p = torch.softmax(log_p / temperature, -1)[0, 0]
            assert (p[tour_tensor] == 0).all()
            assert (p.sum() - 1).abs() < 1e-5
            p = p.numpy()
            next_node = np.argmax(p)
            # print("N children", mcts_20_nodes.root._children.keys())
            # print(p)
            tour.append(next_node)
            
        t_e = time.perf_counter()
        tour.append(0) # Return to the starting position
        tour_len = evaluate_tour(graph.numpy(), tour)
        results.append(tour_len)
#         best_seen_results.append(mcts_20_nodes.best_seen_length)
        timestamps.append(t_e - t_s)
#         print("\nGraph", i)
#         print(evaluate_tour(graph.numpy(), tour))
#         print("Best tour seen", mcts_20_nodes.best_seen_length)

results_dict = {
    "result" : results,
    "timestamps" : timestamps
}

save_string = F"experiments/benchmark_greedy.pkl"
# print(save_string)
with open(save_string, "wb") as f:
    pickle.dump(results_dict, f)


In [4]:
total_len = 0
total_len_best_seen = 0

mcts_timestamps = []
mcts_results = []
best_seen_results = []

for i in range(n_graphs):
    graph = graphs[i]
    graph_batch = graph[None] # Add batch dimension
    tour = [0] # Start at first node, unconventional, TODO: fix this
    mcts_20_nodes = MCTS_TSP(graph.numpy(), 0, n_expansions, n_rollouts, eval_selection="best", eval_rollout="mean")
    t_s = time.perf_counter()
    with torch.no_grad():
        embeddings, _ = model.embedder(model._init_embed(graph_batch))

        # Compute keys, values for the glimpse and keys for the logits once as they can be reused in every step
        fixed = model._precompute(embeddings)
        for visit in range(graph.shape[0] - 1):
            tour_tensor = torch.tensor(tour).long()
            if len(tour_tensor) == 0:
                step_context = model.W_placeholder
            else:
                step_context = torch.cat((embeddings[0, tour_tensor[0]],
                                        embeddings[0, tour_tensor[-1]]), -1)
            query = fixed.context_node_projected + model.project_step_context(step_context[None, None, :])
            mask = torch.zeros(graph_batch.shape[1], dtype=torch.uint8) > 0
            mask[tour_tensor] = 1
            mask = mask[None, None, :]

            log_p, _ = model._one_to_many_logits(query, fixed.glimpse_key, fixed.glimpse_val, fixed.logit_key, mask)
            p = torch.softmax(log_p / temperature, -1)[0, 0]
            assert (p[tour_tensor] == 0).all()
            assert (p.sum() - 1).abs() < 1e-5
            p = p.numpy()
            mcts_20_nodes.update_priors(p)
            next_node = mcts_20_nodes.mcts_decide()
            # print("N children", mcts_20_nodes.root._children.keys())
            # print(p)
            tour.append(next_node)
            mcts_20_nodes.move_to(next_node)
        t_e = time.perf_counter()
        tour.append(0) # Return to the starting position
        tour_len = evaluate_tour(graph.numpy(), tour)
        mcts_results.append(tour_len)
        best_seen_results.append(mcts_20_nodes.best_seen_length)
        mcts_timestamps.append(t_e - t_s)
        print("\nGraph", i)
        print(evaluate_tour(graph.numpy(), tour))
        print("Best tour seen", mcts_20_nodes.best_seen_length)


Graph 0
4.8627996
Best tour seen 4.727939

Graph 1
5.3296275
Best tour seen 5.174877

Graph 2
5.0051584
Best tour seen 4.987064

Graph 3
5.063277
Best tour seen 5.063277

Graph 4
4.360171
Best tour seen 4.2643023

Graph 5
4.9554367
Best tour seen 4.9554367

Graph 6
5.14106
Best tour seen 4.830803

Graph 7
4.5489144
Best tour seen 4.5489144

Graph 8
5.4290514
Best tour seen 5.4290514

Graph 9
4.7659144
Best tour seen 4.765914


In [6]:
print("Average MCTS results", np.mean(mcts_results))
print("Average best seen results", np.mean(best_seen_results))
print("Average duration", np.mean(mcts_timestamps))

Average MCTS results 4.946141
Average best seen results 4.874758
Average duration 7.104339689999999


In [ ]:
results_dict = {
    "mcts_result" : mcts_results,
    "best_seen_results" : best_seen_results,
    "mcts_timestamps" : mcts_timestamps
}

In [ ]:
save_string = F"experiments/{n_nodes}_{n_graphs}_{n_expansions}_{n_rollouts}_{eval_selection}_{eval_rollout}.pkl"
# print(save_string)
with open(save_string, "wb") as f:
    pickle.dump(results_dict, f)

experiments/20_10_100_100_best_best.pkl


# Use this for experiment loops

In [16]:
### PARAMETERS TO BE SET ###
n_graphs = 100
n_nodes = 20

n_expansions = 100
n_rollouts = 100
eval_selection = "mean"
eval_rollout = "mean"

temperature = 1


# Load model 20 nodes and set temperature
model, _ = load_model(F'pretrained/tsp_{n_nodes}/')
model.eval()
print("model loaded")
# Load test set graphs 20 nodes
# If this block does not work, make sure you called:
# python generate_data.py --problem all --name test --seed 1234
with open(F"data/tsp/tsp{n_nodes}_test_seed1234.pkl", "rb") as f:
    data = pickle.load(f)
    dataset = TSPDataset(None, 0, 0, 0, None)
    dataset.data = [torch.FloatTensor(row) for row in (data[0:0+10000])]
    dataset.size = len(dataset.data)
    graphs = []
    for sample in dataset.data:
        graphs.append(sample)



for n_rollouts in [50, 150, 500]:
    for n_expansions in [50, 150, 500]:
        total_len = 0
        total_len_best_seen = 0

        mcts_timestamps = []
        mcts_results = []
        best_seen_results = []

        for i in range(n_graphs):
            graph = graphs[i]
            graph_batch = graph[None] # Add batch dimension
            tour = [0] # Start at first node, unconventional, TODO: fix this
            mcts_20_nodes = MCTS_TSP(graph.numpy(), 0, n_expansions, n_rollouts, eval_selection=eval_selection, eval_rollout=eval_rollout)
            t_s = time.perf_counter()
            with torch.no_grad():
                embeddings, _ = model.embedder(model._init_embed(graph_batch))

                # Compute keys, values for the glimpse and keys for the logits once as they can be reused in every step
                fixed = model._precompute(embeddings)
                for visit in range(graph.shape[0] - 1):
                    tour_tensor = torch.tensor(tour).long()
                    if len(tour_tensor) == 0:
                        step_context = model.W_placeholder
                    else:
                        step_context = torch.cat((embeddings[0, tour_tensor[0]],
                                                embeddings[0, tour_tensor[-1]]), -1)
                    query = fixed.context_node_projected + model.project_step_context(step_context[None, None, :])
                    mask = torch.zeros(graph_batch.shape[1], dtype=torch.uint8) > 0
                    mask[tour_tensor] = 1
                    mask = mask[None, None, :]

                    log_p, _ = model._one_to_many_logits(query, fixed.glimpse_key, fixed.glimpse_val, fixed.logit_key, mask)
                    p = torch.softmax(log_p / temperature, -1)[0, 0]
                    assert (p[tour_tensor] == 0).all()
                    assert (p.sum() - 1).abs() < 1e-5
                    p = p.numpy()
                    mcts_20_nodes.update_priors(p)
                    next_node = mcts_20_nodes.mcts_decide()
                    # print("N children", mcts_20_nodes.root._children.keys())
                    # print(p)
                    tour.append(next_node)
                    mcts_20_nodes.move_to(next_node)
                t_e = time.perf_counter()
                tour.append(0) # Return to the starting position
                tour_len = evaluate_tour(graph.numpy(), tour)
                mcts_results.append(tour_len)
                best_seen_results.append(mcts_20_nodes.best_seen_length)
                mcts_timestamps.append(t_e - t_s)
                print("\nGraph", i, n_rollouts, n_expansions)
                # print(evaluate_tour(graph.numpy(), tour))
                # print("Best tour seen", mcts_20_nodes.best_seen_length)

        results_dict = {
        "mcts_result" : mcts_results,
        "best_seen_results" : best_seen_results,
        "mcts_timestamps" : mcts_timestamps
        }

        save_string = F"experiments/{n_nodes}_{n_graphs}_{n_expansions}_{n_rollouts}_{eval_selection}_{eval_rollout}.pkl"
        # print(save_string)
        with open(save_string, "wb") as f:
            pickle.dump(results_dict, f)

  [*] Loading model from pretrained/tsp_20/epoch-99.pt
model loaded

Graph 0 50 50

Graph 1 50 50

Graph 2 50 50

Graph 3 50 50

Graph 4 50 50

Graph 5 50 50

Graph 6 50 50

Graph 7 50 50

Graph 8 50 50

Graph 9 50 50

Graph 10 50 50

Graph 11 50 50

Graph 12 50 50

Graph 13 50 50

Graph 14 50 50

Graph 15 50 50

Graph 16 50 50

Graph 17 50 50

Graph 18 50 50

Graph 19 50 50

Graph 20 50 50

Graph 21 50 50

Graph 22 50 50

Graph 23 50 50

Graph 24 50 50

Graph 25 50 50

Graph 26 50 50

Graph 27 50 50

Graph 28 50 50

Graph 29 50 50

Graph 30 50 50

Graph 31 50 50

Graph 32 50 50

Graph 33 50 50

Graph 34 50 50

Graph 35 50 50

Graph 36 50 50

Graph 37 50 50

Graph 38 50 50

Graph 39 50 50

Graph 40 50 50

Graph 41 50 50

Graph 42 50 50

Graph 43 50 50

Graph 44 50 50

Graph 45 50 50

Graph 46 50 50

Graph 47 50 50

Graph 48 50 50

Graph 49 50 50

Graph 50 50 50

Graph 51 50 50

Graph 52 50 50

Graph 53 50 50

Graph 54 50 50

Graph 55 50 50

Graph 56 50 50

Graph 57 50 50

Graph 58 50 5

In [3]:
import os
import numpy as np
import pickle
directory = "experiments"

results = {}

for f in os.listdir(directory):
    file = os.path.join(directory, f)
    if os.path.isfile(file):
        with open(file, "rb") as handle:
            results[f] = pickle.load(handle)

In [6]:
for i in results:
#     print(i)
    if "greedy" not in i or not "beam" in i:
        mcts_mean_std = np.around((np.mean(results[i]['mcts_result']), np.std(results[i]['mcts_result'])), 3)
        best_seen_mean_std = np.around((np.mean(results[i]['best_seen_results']), np.std(results[i]['best_seen_results'])), 3)
        time_mean_std = np.around((np.mean(results[i]['mcts_timestamps']), np.std(results[i]['mcts_timestamps'])), 3)
#         print(i[:-4], F"\tmcts {mcts_mean_std[0]} \tstd {mcts_mean_std[1]} \t\tbest {best_seen_mean_std[0]} \tstd {best_seen_mean_std[1]} \t\ttime {time_mean_std[0]} \t std {time_mean_std[1]}")
        print(i, "\t", mcts_mean_std, best_seen_mean_std, time_mean_std)
    else:
        greedy_mean_std = np.around((np.mean(results[i]['result']), np.std(results[i]['result'])), 3)
        greedy_time_mean_std = np.around((np.mean(results[i]['timestamps']), np.std(results[i]['timestamps'])), 3)
        print(i, "\t\t", greedy_mean_std, "\t\t    ", greedy_time_mean_std)


    
#     print(i, np.mean(results[i]['mcts_result']), np.std(results[i]['mcts_result']))


# for i in results:
#     print(i, np.mean(results[i]['best_seen']), np.std(results[i]['best_seen']))

100_100_100_100_mean_mean.pkl 	 [14.566  0.859] [14.504  0.848] [33.327  1.18 ]
20_100_100_100_best_best.pkl 	 [5.679 0.671] [5.585 0.619] [4.996 0.114]
20_100_100_100_best_mean.pkl 	 [4.851 0.513] [4.685 0.484] [4.992 0.111]
20_100_100_100_mean_best.pkl 	 [5.043 0.637] [4.781 0.51 ] [5.058 0.089]
20_100_100_100_mean_mean.pkl 	 [4.373 0.437] [4.238 0.398] [5.119 0.362]
20_100_150_150_mean_mean.pkl 	 [4.296 0.452] [4.166 0.402] [10.005  0.398]
20_100_150_500_mean_mean.pkl 	 [4.307 0.404] [4.125 0.35 ] [30.427  1.835]
20_100_150_50_mean_mean.pkl 	 [4.317 0.446] [4.188 0.379] [4.458 0.251]
20_100_500_150_mean_mean.pkl 	 [4.197 0.38 ] [4.045 0.318] [31.64   1.361]
20_100_500_500_mean_mean.pkl 	 [4.169 0.374] [4.034 0.318] [95.81   2.612]
20_100_500_50_mean_mean.pkl 	 [4.203 0.36 ] [4.061 0.317] [13.678  0.516]
20_100_50_150_mean_mean.pkl 	 [4.537 0.461] [4.399 0.412] [3.867 0.119]
20_100_50_500_mean_mean.pkl 	 [4.532 0.445] [4.355 0.394] [10.818  0.437]
20_100_50_50_mean_mean.pkl 	 [4.582 

KeyError: 'mcts_result'